In [15]:
def group_bounding_boxes_per_frame(labels_file, output_file, iou_threshold=0.5):
    """
    Groups overlapping bounding boxes of the same class into a single bounding box per frame.
    Uses one of the track IDs from the grouped set for the resulting bounding box.

    Parameters:
    labels_file (str): Path to the labels.txt file containing bounding box information.
    output_file (str): Path to save the updated labels file with grouped bounding boxes.
    iou_threshold (float): IoU threshold to consider two boxes as overlapping.
    """
    def calculate_iou(box1, box2):
        # Calculate intersection coordinates
        x1_inter = max(box1[0], box2[0])
        y1_inter = max(box1[1], box2[1])
        x2_inter = min(box1[2], box2[2])
        y2_inter = min(box1[3], box2[3])

        # Calculate intersection area
        inter_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)

        # Calculate union area
        box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
        box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union_area = box1_area + box2_area - inter_area

        # Return IoU
        return inter_area / union_area if union_area > 0 else 0

    # Read labels file and extract data
    with open(labels_file, 'r') as file:
        labels = [line.strip().split() for line in file]

    # Group by frame
    frames = {}
    for line in labels:
        frame = int(line[0])
        bbox = list(map(float, line[6:10]))
        obj_data = line[:6] + [bbox] + line[10:]
        if frame not in frames:
            frames[frame] = []
        frames[frame].append(obj_data)

    # Process each frame for grouping
    updated_labels = []
    for frame, objects in frames.items():
        grouped = []
        used = set()

        for i, obj1 in enumerate(objects):
            if i in used:
                continue

            # Start a new group
            group = [obj1]
            used.add(i)

            for j, obj2 in enumerate(objects):
                if j in used or obj1[2] != obj2[2]:  # Skip if already grouped or different class
                    continue

                # Calculate IoU
                bbox1 = obj1[6]
                bbox2 = obj2[6]
                iou = calculate_iou(bbox1, bbox2)

                if iou >= iou_threshold:
                    group.append(obj2)
                    used.add(j)

            grouped.append(group)

        # Merge grouped bounding boxes and update labels
        for group in grouped:
            if len(group) == 1:
                # Single box, no grouping needed
                updated_labels.append(" ".join(map(str, group[0][:6] + group[0][6] + group[0][7:])))
            else:
                # Merge bounding boxes
                min_x = min(obj[6][0] for obj in group)
                min_y = min(obj[6][1] for obj in group)
                max_x = max(obj[6][2] for obj in group)
                max_y = max(obj[6][3] for obj in group)
                merged_bbox = [min_x, min_y, max_x, max_y]

                # Use the first object's data for the new bounding box
                representative = group[0]
                merged_obj = representative[:6] + merged_bbox + representative[10:]
                updated_labels.append(" ".join(map(str, merged_obj)))

    # Write updated labels to output file
    with open(output_file, 'w') as file:
        file.writelines(label + "\n" for label in updated_labels)

    print(f"Grouped bounding boxes saved to {output_file}")


In [14]:
group_bounding_boxes_per_frame(
    labels_file=r'C:\Users\User\Documents\GitHub\pfas_finalproject\Sequence Labels\Seq_02.txt',
    output_file=r'C:\Users\User\Documents\GitHub\pfas_finalproject\kalman\labels_grouped.txt',
    iou_threshold=0.3  # Specify IoU threshold
)


Grouped bounding boxes saved to C:\Users\User\Documents\GitHub\pfas_finalproject\kalman\labels_grouped.txt


In [1]:
def create_labels_with_removed_x_range(labels_file, x_range, new_labels_file):
    """
    Creates a new labels file with track IDs removed if their bounding box falls within a specified x-coordinate range.

    Parameters:
    labels_file (str): Path to the original labels file.
    x_range (tuple): A tuple (x_min, x_max) specifying the range of x-coordinates.
    new_labels_file (str): Path to save the new labels file with track IDs removed.

    Returns:
    None
    """
    x_min, x_max = x_range

    # Read original labels
    with open(labels_file, 'r') as file:
        labels = file.readlines()

    # Prepare modified data
    modified_labels = []
    for line in labels:
        columns = line.strip().split()
        bbox = list(map(float, columns[6:10]))  # Extract bounding box: [left, top, right, bottom]
        left, _, right, _ = bbox

        # Check if the bounding box overlaps with the specified x range
        if x_min <= left <= x_max or x_min <= right <= x_max:
            continue  # Skip this line if the bounding box falls in the x range

        # Add line to modified labels if it doesn't match the exclusion criteria
        modified_labels.append(line)

    # Write modified labels to new file
    with open(new_labels_file, 'w') as file:
        file.writelines(modified_labels)

    print(f"New labels file created: {new_labels_file}")


In [4]:
create_labels_with_removed_x_range(
    labels_file=r'C:\Users\User\Documents\dataset\pfas\34759_final_project_rect\seq_02\labels.txt',
    x_range=(660, 910),
    new_labels_file=r'C:\Users\User\Documents\GitHub\pfas_finalproject\kalman\artificial_occlusion_better_02.txt'
)

New labels file created: C:\Users\User\Documents\GitHub\pfas_finalproject\kalman\artificial_occlusion_better_02.txt
